In [1]:
!pip install bitsandbytes
!pip install -q datasets loralib sentencepiece
!pip install -q git+https://github.com/zphang/transformers@c3dc391
!pip install -q git+https://github.com/huggingface/peft.git


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 19.5 MB/s eta 0:00:00
  Installing b

In [2]:
!pip -q install configparser langchain
!pip -q install transformers huggingface_hub 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.5/408.5 KB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 KB 5.6 MB/s eta 0:00:00


In [3]:
import configparser
import os

reader = configparser.ConfigParser()
reader.read_file(open('/content/apidata.config'))

os.environ['SERPAPI_API_KEY']=reader["OPENAI"]["SERPAPI_KEY"]
os.environ['SERPER_API_KEY']=reader["OPENAI"]["SERPER_KEY"]
os.environ['HUGGINGFACEHUB_API_TOKEN'] = reader["OPENAI"]["HFREAD"]

In [ ]:
"""Initiating the Wrapper around HuggingFace Pipeline APIs."""
import importlib.util
from typing import Any, List, Mapping, Optional
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig
from pydantic import BaseModel, Extra

from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens


In [5]:
DEFAULT_MODEL_ID = "llama-7b-hf"
DEFAULT_TASK = "text-generation"
VALID_TASKS = ("text2text-generation", "text-generation")


In [6]:
class HuggingFacePipeline(LLM, BaseModel):
    """Wrapper around HuggingFace Pipeline API.

    To use, you should have the ``transformers`` python package installed.

    Only supports `text-generation` and `text2text-generation` for now.

    Example using from_model_id:
        .. code-block:: python

            from langchain.llms import HuggingFacePipeline
            hf = HuggingFacePipeline.from_model_id(
                model_id="gpt2", task="text-generation"
            )
    Example passing pipeline in directly:
        .. code-block:: python

            from langchain.llms import HuggingFacePipeline
            from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

            model_id = "gpt2"
            tokenizer = AutoTokenizer.from_pretrained(model_id)
            model = AutoModelForCausalLM.from_pretrained(model_id)
            pipe = pipeline(
                "text-generation", model=model, tokenizer=tokenizer, max_new_tokens=10
            )
            hf = HuggingFacePipeline(pipeline=pipe)
    """

    pipeline: Any  #: :meta private:
    model_id: str = DEFAULT_MODEL_ID
    """Model name to use."""
    model_kwargs: Optional[dict] = None
    """Key word arguments to pass to the model."""

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid

    @classmethod
    def from_model_id(
        cls,
        model_id: str,
        task: str,
        device: int = -1,
        model_kwargs: Optional[dict] = None,
        **kwargs: Any,
    ) -> LLM:
        """Construct the pipeline object from model_id and task."""
        try:
            from transformers import (
                LLaMAForCausalLM,
                LLaMATokenizer,
            )
            from transformers import pipeline as hf_pipeline

        except ImportError:
            raise ValueError(
                "Could not import transformers python package. "
                "Please it install it with `pip install transformers`."
            )

        _model_kwargs = model_kwargs or {}
        tokenizer = LLaMATokenizer.from_pretrained(model_id, **_model_kwargs)

        try:
            if task == "text-generation":
                model = LLaMAForCausalLM.from_pretrained(model_id,
                                                         load_in_8bit=True,
                                                         device_map='auto')
            else:
                raise ValueError(
                    f"Got invalid task {task}, "
                    f"currently only {VALID_TASKS} are supported"
                )
        except ImportError as e:
            raise ValueError(
                f"Could not load the {task} model due to missing dependencies."
            ) from e

        if importlib.util.find_spec("torch") is not None:
            import torch

            cuda_device_count = torch.cuda.device_count()
            if device < -1 or (device >= cuda_device_count):
                raise ValueError(
                    f"Got device=={device}, "
                    f"device is required to be within [-1, {cuda_device_count})"
                )
            if device < 0 and cuda_device_count > 0:
                print(
                    "Device has %d GPUs available. "
                    "Provide device={deviceId} to `from_model_id` to use available"
                    "GPUs for execution. deviceId is -1 (default) for CPU and "
                    "can be a positive integer associated with CUDA device id.",
                    cuda_device_count,
                )

        pipeline = hf_pipeline(
            task=task,
            model=model,
            tokenizer=tokenizer,
            device=device,
            model_kwargs=_model_kwargs,
        )
        if pipeline.task not in VALID_TASKS:
            raise ValueError(
                f"Got invalid task {pipeline.task}, "
                f"currently only {VALID_TASKS} are supported"
            )
        return cls(
            pipeline=pipeline,
            model_id=model_id,
            model_kwargs=_model_kwargs,
            **kwargs,
        )

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {
            **{"model_id": self.model_id},
            **{"model_kwargs": self.model_kwargs},
        }

    @property
    def _llm_type(self) -> str:
        return "huggingface_pipeline"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        response = self.pipeline(prompt)
        if self.pipeline.task == "text-generation":
            # Text generation return includes the starter text.
            text = response[0]["generated_text"][len(prompt) :]
        elif self.pipeline.task == "text2text-generation":
            text = response[0]["generated_text"]
        else:
            raise ValueError(
                f"Got invalid task {self.pipeline.task}, "
                f"currently only {VALID_TASKS} are supported"
            )
        if stop is not None:
            # This is a bit hacky, but I can't figure out a better way to enforce
            # stop tokens when making calls to huggingface_hub.
            text = enforce_stop_tokens(text, stop)
        return text

In [ ]:
#from langchain.llms import HuggingFacePipeline

#hf_fmd = HuggingFacePipeline.from_model_id(model_id="decapoda-research/llama-7b-hf", task="text-generation")

In [ ]:
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig

model_id = "decapoda-research/llama-7b-hf"
tokenizer = LLaMATokenizer.from_pretrained(model_id)


In [8]:
model = LLaMAForCausalLM.from_pretrained(model_id,
                                         load_in_8bit=True,
                                         device_map='auto')

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [9]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
pipe = pipeline(
    "text-generation", model=model, 
    tokenizer=tokenizer, 
    max_new_tokens=1000
)

In [10]:
hf_fullpipe = HuggingFacePipeline(pipeline=pipe)

In [11]:
print(hf_fullpipe)

HuggingFacePipeline
Params: {'model_id': 'gpt2', 'model_kwargs': None}


In [13]:
# Trying out the prompt templates with the Alpaca Lora
from langchain import PromptTemplate

product_prompt = PromptTemplate(
    input_variables=["question"], 
    template="Answer the above question on {question}."
)
product_prompt.format(question='Laptop')

'Answer the above question on Laptop.'

In [14]:
from langchain.chains import LLMChain

product_chain = LLMChain(llm=hf_fullpipe,prompt=product_prompt)

In [15]:
product_chain.run("Who are you?")

KeyboardInterrupt: ignored

In [ ]:
product_chain.run("Dell")

KeyboardInterrupt: ignored

In [ ]:
hf_llm=HuggingFaceHub(repo_id="google/flan-t5-xl",
                             model_kwargs={"temperature":0,
                                           "max_length":64})

alpaca_llm=HuggingFaceHub(repo_id="decapoda-research/llama-7b-hf",
                             model_kwargs={"temperature":0,
                                           "max_length":64})

In [ ]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

template = """Question: {question}

Answer: Let's think step by step."""

prompt_hf_hub = PromptTemplate(template=template, input_variables=["question"])

llm_chain = LLMChain(prompt=prompt_hf_hub,llm = hf_llm)

In [ ]:
question = "What NFL team won the Super Bowl in the year Justin Beiber was born?"

print(llm_chain.run(question))

The Seattle Seahawks won the Super Bowl in 2010. Justin Beiber was born in 2010. The final answer: Seattle Seahawks.


In [ ]:
alpaca_chain = LLMChain(prompt=prompt_hf_hub,llm = alpaca_llm)

In [ ]:
alpaca_chain.run(question)

ValueError: ignored